# <a id='model-selection-and-evaluation'></a> 3. [**Sélection et évaluation de modèle**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#model-selection-and-evaluation)</br>([*Model selection and evaluation*](https://scikit-learn.org/stable/model_selection.html#model-selection-and-evaluation))

# 3.2. [**Réglage des hyper-paramètres d'un estimateur**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#tuning-the-hyper-parameters-of-an-estimator)<br/>([_Tuning the hyper-parameters of an estimator_](https://scikit-learn.org/stable/model_selection.html#tuning-the-hyper-parameters-of-an-estimator))

# Sommaire

- **Volume** : 18 pages, 9 exemples, 3 papiers
- 3.2.1. [**Recherche exhaustive en grille**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#exhaustive-grid-search)<br/>([_Exhaustive Grid Search_](https://scikit-learn.org/stable/model_selection.html#exhaustive-grid-search))
- 3.2.2. [**Optimisation aléatoire des paramètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#randomized-parameter-optimization)<br/>([_Randomized Parameter Optimization_](https://scikit-learn.org/stable/model_selection.html#randomized-parameter-optimization))
- 3.2.3. [**Recherche de paramètres optimaux par halving successifs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#searching-for-optimal-parameters-with-successive-halving)<br/>([_Searching for optimal parameters with successive halving_](https://scikit-learn.org/stable/model_selection.html#searching-for-optimal-parameters-with-successive-halving))
- 3.2.4. [**Conseils pour la recherche de paramètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#tips-for-parameter-search)<br/>([_Tips for parameter search_](https://scikit-learn.org/stable/model_selection.html#tips-for-parameter-search))
- 3.2.5. [**Alternatives à la recherche de paramètres par force brute**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#alternatives-to-brute-force-parameter-search)<br/>([_Alternatives to brute force parameter search_](https://scikit-learn.org/stable/model_selection.html#alternatives-to-brute-force-parameter-search))

Les hyperparamètres sont des paramètres qui ne sont pas directement appris au sein des estimateurs. Dans scikit-learn, ils sont transmis en tant qu'arguments au constructeur des classes d'estimateur. Des exemples typiques incluent `C`, `kernel` et `gamma` pour le classifieur à vecteurs de support, `alpha` pour Lasso, etc.

Il est possible et recommandé d'explorer l'espace des hyperparamètres pour obtenir le meilleur score de [**validation croisée** (3.1)](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation).

Tout paramètre fourni lors de la construction d'un estimateur peut être optimisé de cette manière. Pour trouver les noms et les valeurs actuelles de tous les paramètres pour un estimateur donné, utilisez :

```python
estimator.get_params()
```

Une recherche comprend :
- un estimateur (régresseur ou classifieur tel que `sklearn.svm.SVC()`);
- un espace de paramètres ;
- une méthode pour rechercher ou échantillonner des candidats ;
- un schéma de validation croisée ; et
- une [**fonction de score** (3.2.4.1)](https://scikit-learn.org/stable/modules/grid_search.html#gridsearch-scoring).

Deux approches génériques de recherche de paramètres sont fournies dans scikit-learn : pour des valeurs données, [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) considère exhaustivement toutes les combinaisons de paramètres, tandis que [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) peut échantillonner un nombre donné de candidats dans un espace de paramètres avec une distribution spécifiée. Ces deux outils ont des contreparties successives, [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) et [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html), qui peuvent être beaucoup plus rapides pour trouver une bonne combinaison de paramètres.

Après avoir décrit ces outils, nous détaillons les [**meilleures pratiques** (3.2.4)](https://scikit-learn.org/stable/modules/grid_search.html#grid-search-tips) applicables à ces approches. Certains modèles permettent des stratégies de recherche de paramètres spécialisées et efficaces, décrites dans [**Alternatives to brute force parameter search** (3.2.5)](https://scikit-learn.org/stable/modules/grid_search.html#alternative-cv).

Notez qu'il est courant qu'un petit sous-ensemble de ces paramètres puisse avoir un impact important sur les performances prédictives ou calculatoires du modèle, tandis que d'autres peuvent être laissés à leurs valeurs par défaut. Il est recommandé de lire la documentation de la classe d'estimateur pour mieux comprendre leur comportement attendu, éventuellement en lisant la référence contenue dans la documentation.

## <a id='exhaustive-grid-search'></a> 3.2.1. Recherche exhaustive en grille

La recherche en grille fournie par [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) génère de manière exhaustive des candidats à partir d'une grille de valeurs de paramètres spécifiée avec le paramètre `param_grid`. Par exemple, la grille suivante :

In [1]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

L'instance [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) implémente l'API habituelle de l'estimateur : lorsqu'elle est "ajustée" sur un ensemble de données, toutes les combinaisons possibles de valeurs de paramètres sont évaluées et la meilleure combinaison est conservée.

### Exemples

#### [**Stratégie personnalisée de réajustement d'une recherche en grille avec validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_digits.ipynb)<br/>([_Custom refit strategy of a grid search with cross-validation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html))

Un exemple de calcul de recherche en grille sur l'ensemble de données "digits".

#### [**Exemple de pipeline pour l'extraction et l'évaluation de caractéristiques de texte**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_text_feature_extraction.ipynb)<br/>([_Sample pipeline for text feature extraction and evaluation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_text_feature_extraction.html))
Un exemple d'utilisation de la recherche en grille couplant des paramètres d'un extracteur de caractéristiques de documents texte (comptage des n-grammes et transformation TF-IDF) avec un classifieur (ici un SVM linéaire entraîné avec SGD avec soit une régularisation de type elastic net, soit une régularisation de type L2) à l'aide d'une instance `pipeline.Pipeline`.

#### [**Validation croisée imbriquée vs. non-imbriquée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_nested_cross_validation_iris.ipynb)<br/>([_Nested versus non-nested cross-validation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html))

Un exemple de recherche en grille dans une boucle de validation croisée sur l'ensemble de données "Iris". Il s'agit de la meilleure pratique pour évaluer les performances d'un modèle avec la recherche en grille.

#### [**Démonstration de l'évaluation multi-métrique sur `cross_val_score` et `GridSearchCV`**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_multi_metric_evaluation.ipynb)<br/>([_Demonstration of multi-metric evaluation on `cross_val_score` and `GridSearchCV`_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html))

Un exemple d'utilisation de [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) pour évaluer simultanément plusieurs métriques.

#### [**Équilibrer la complexité du modèle et le score validé par validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_refit_callable.ipynb)<br/>([_Balance model complexity and cross-validated score_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_refit_callable.html))

Un exemple d'utilisation de l'interface `refit=callable` dans [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). L'exemple montre comment cette interface ajoute une certaine flexibilité pour identifier le meilleur estimateur. Cette interface peut également être utilisée dans l'évaluation de plusieurs métriques.

#### [**Comparaison statistique des modèles à l'aide de la recherche en grille**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_stats.ipynb)<br/>([_Statistical comparison of models using grid search_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_stats.html))

Un exemple de comparaison statistique des résultats de [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

## <a id='randomized-parameter-optimization'></a> 3.2.2. Optimisation de paramètres aléatoire

Bien que l'utilisation d'une grille de paramètres soit actuellement la méthode la plus largement utilisée pour l'optimisation des paramètres, d'autres méthodes de recherche présentent des propriétés plus favorables. [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) met en œuvre une recherche aléatoire des paramètres, où chaque paramètre est échantillonné à partir d'une distribution de valeurs de paramètres possibles. Cela présente deux avantages principaux par rapport à une recherche exhaustive :

- Un budget peut être choisi indépendamment du nombre de paramètres et des valeurs possibles.
- L'ajout de paramètres qui n'influencent pas les performances n'affecte pas l'efficacité.

La spécification de la manière dont les paramètres doivent être échantillonnés est effectuée à l'aide d'un dictionnaire, très similaire à la spécification des paramètres pour [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). De plus, un budget de calcul, représenté par le nombre de candidats échantillonnés ou d'itérations d'échantillonnage, est spécifié à l'aide du paramètre `n_iter`. Pour chaque paramètre, il est possible de spécifier soit une distribution de valeurs possibles, soit une liste de choix discrets (qui seront échantillonnés uniformément) :

In [ ]:
import scipy
{
    'C': scipy.stats.expon(scale=100),
    'gamma': scipy.stats.expon(scale=.1),
    'kernel': ['rbf'],
    'class_weight':['balanced', None]
}

Cet exemple utilise le module `scipy.stats`, qui contient de nombreuses distributions utiles pour l'échantillonnage des paramètres, telles que `expon`, `gamma`, `uniform`, `loguniform` ou `randint`.

En principe, n'importe quelle fonction peut être utilisée, pourvu qu'elle fournisse une méthode `rvs` (échantillonnage de variable aléatoire) pour échantillonner une valeur. Un appel à la fonction `rvs` doit fournir des échantillons aléatoires indépendants à partir des valeurs de paramètres possibles lors des appels consécutifs.

> **Attention:** Les distributions dans `scipy.stats` avant la version scipy 0.16 ne permettent pas de spécifier un état aléatoire. À la place, elles utilisent l'état aléatoire global de numpy, qui peut être initialisé via `np.random.seed` ou défini à l'aide de `np.random.set_state`. Cependant, à partir de scikit-learn 0.18, le module [**`sklearn.model_selection`**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) définira l'état aléatoire fourni par l'utilisateur si scipy >= 0.16 est également disponible.

Pour les paramètres continus, tels que `C` ci-dessus, il est important de spécifier une distribution continue pour tirer pleinement parti de la randomisation. De cette façon, l'augmentation de `n_iter` conduira toujours à une recherche plus fine.

Une variable aléatoire continue uniforme logarithmique est la version continue d'un paramètre réparti logarithmiquement. Par exemple, pour spécifier l'équivalent de `C` ci-dessus, on peut utiliser `loguniform(1, 100)` au lieu de `[1, 10, 100]`.

En suivant l'exemple ci-dessus de recherche en grille, nous pouvons spécifier une variable aléatoire continue répartie de manière logarithmique entre `1e0` et `1e3` :

In [ ]:
from sklearn.utils.fixes import loguniform
{
    'C': loguniform(1e0, 1e3),
    'gamma': loguniform(1e-4, 1e-3),
    'kernel': ['rbf'],
    'class_weight':['balanced', None]
}

### Exemples

#### [**Comparaison de la recherche aléatoire et de la recherche en grille pour l'estimation d'hyperparamètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_randomized_search.ipynb)<br/>([_Comparing randomized search and grid search for hyperparameter estimation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html))

Compare l'utilisation et l'efficacité de la recherche aléatoire et de la recherche en grille.

### Références

🔬 Bergstra, J. and Bengio, Y., [**“Random search for hyper-parameter optimization”**](https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf), The Journal of Machine Learning Research (2012)

## <a id='searching-for-optimal-parameters-with-successive-halving'></a> 3.2.3. Searching for optimal parameters with successive halving

Scikit-learn also provides the [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) and [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) estimators that can be used to search a parameter space using successive halving [1] [2]. Successive halving (SH) is like a tournament among candidate parameter combinations. SH is an iterative selection process where all candidates (the parameter combinations) are evaluated with a small amount of resources at the first iteration. Only some of these candidates are selected for the next iteration, which will be allocated more resources. For parameter tuning, the resource is typically the number of training samples, but it can also be an arbitrary numeric parameter such as `n_estimators` in a random forest.

As illustrated in the figure below, only a subset of candidates ‘survive’ until the last iteration. These are the candidates that have consistently ranked among the top-scoring candidates across all iterations. Each iteration is allocated an increasing amount of resources per candidate, here the number of samples.

<div style="text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/sphx_glr_plot_successive_halving_iterations_001.png"
    alt="Flux de travail de recherche en grille"
    style="max-width: 50%; height: auto;">
</div>

We here briefly describe the main parameters, but each parameter and their interactions are described in more details in the sections below. The `factor` (> 1) parameter controls the rate at which the resources grow, and the rate at which the number of candidates decreases. In each iteration, the number of resources per candidate is multiplied by `factor` and the number of candidates is divided by the same factor. Along with `resource` and `min_resources`, factor is the most important parameter to control the search in our implementation, though a value of 3 usually works well. `factor` effectively controls the number of iterations in [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) and the number of candidates (by default) and iterations in [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html). `aggressive_elimination=True` can also be used if the number of available resources is small. More control is available through tuning the `min_resources` parameter.

These estimators are still experimental: their predictions and their API might change without any deprecation cycle. To use them, you need to explicitly import `enable_halving_search_cv`:

```python
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv  # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV
```

### Examples

#### Comparison between grid search and successive halving
#### Successive Halving Iterations

## <a id='searching-for-optimal-parameters-with-successive-halving'></a> 3.2.3. Recherche des paramètres optimaux avec le "Successive Halving"

Scikit-learn propose également les estimateurs [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) et [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) qui peuvent être utilisés pour rechercher dans un espace de paramètres en utilisant le "Successive Halving" [1] [2]. Le "Successive Halving" (SH) est comme un tournoi entre les combinaisons de paramètres candidates. Le SH est un processus de sélection itératif où toutes les combinaisons candidates (les combinaisons de paramètres) sont évaluées avec une petite quantité de ressources lors de la première itération. Seules certaines de ces combinaisons candidates sont sélectionnées pour la prochaine itération, qui bénéficiera de plus de ressources. Pour l'optimisation des paramètres, la ressource est généralement le nombre d'échantillons d'entraînement, mais elle peut également être un paramètre numérique arbitraire tel que `n_estimators` dans une forêt aléatoire.

Comme illustré dans la figure ci-dessous, seuls certains candidats "survivent" jusqu'à la dernière itération. Ce sont les candidats qui se sont constamment classés parmi les meilleurs candidats en termes de score lors de toutes les itérations. À chaque itération, une quantité croissante de ressources par candidat est allouée, ici le nombre d'échantillons.

<div style="text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/sphx_glr_plot_successive_halving_iterations_001.png"
    alt="Flux de travail de recherche en grille"
    style="max-width: 50%; height: auto;">
</div>

Nous décrivons brièvement ici les principaux paramètres, mais chaque paramètre et leurs interactions sont décrits plus en détail dans les sections suivantes. Le paramètre `factor` (> 1) contrôle le taux de croissance des ressources et le taux de diminution du nombre de candidats. À chaque itération, le nombre de ressources par candidat est multiplié par `factor` et le nombre de candidats est divisé par le même facteur. Avec `resource` et `min_resources`, le paramètre `factor` est le paramètre le plus important pour contrôler la recherche dans notre implémentation, bien qu'une valeur de 3 fonctionne généralement bien. `factor` contrôle efficacement le nombre d'itérations dans [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) et le nombre de candidats (par défaut) et d'itérations dans [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html). `aggressive_elimination=True` peut également être utilisé si le nombre de ressources disponibles est faible. Un contrôle plus précis est possible en ajustant le paramètre `min_resources`.

Ces estimateurs sont encore **expérimentaux** : leurs prédictions et leur API peuvent changer sans aucun cycle de dépréciation. Pour les utiliser, vous devez importer explicitement `enable_halving_search_cv` :

In [2]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv  # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

### Exemples

#### [**Comparaison entre la recherche en grille et le "Successive Halving"**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_successive_halving_heatmap.ipynb)<br/>([_Comparison between grid search and successive halving_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_successive_halving_heatmap.html))

#### [**Itérations du "Successive Halving"**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_successive_halving_iterations.ipynb)<br/>([_Successive Halving Iterations_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_successive_halving_iterations.html))

### <a id='choosing-min-resources-and-the-number-of-candidates'></a> 3.2.3.1. Choix de `min_resources` et du nombre de candidats

Outre le paramètre `factor`, les deux principaux paramètres qui influencent le comportement d'une recherche "successive halving" sont le paramètre `min_resources` et le nombre de candidats (ou combinaisons de paramètres) évalués. `min_resources` correspond à la quantité de ressources allouées à chaque candidat lors de la première itération. Le nombre de candidats est spécifié directement dans [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) et est déterminé à partir du paramètre `param_grid` de [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html).

Considérons un cas où la ressource est le nombre d'échantillons et où nous avons 1000 échantillons. En théorie, avec `min_resources=10` et `factor=2`, nous pouvons effectuer **au maximum** 7 itérations avec le nombre d'échantillons suivant : `[10, 20, 40, 80, 160, 320, 640]`.

Cependant, en fonction du nombre de candidats, nous pourrions exécuter moins de 7 itérations : si nous commençons avec un nombre **réduit** de candidats, la dernière itération pourrait utiliser moins de 640 échantillons, ce qui signifie que toutes les ressources disponibles (échantillons) ne sont pas utilisées. Par exemple, si nous commençons avec 5 candidats, nous n'avons besoin que de 2 itérations : 5 candidats pour la première itération, puis `5 // 2 = 2` candidats à la deuxième itération, après quoi nous saurons quel candidat est le meilleur (donc nous n'avons pas besoin d'une troisième itération). Nous n'utiliserions que 20 échantillons au maximum, ce qui serait une perte car nous avons 1000 échantillons à disposition. D'autre part, si nous commençons avec un nombre **élevé** de candidats, nous pourrions nous retrouver avec beaucoup de candidats lors de la dernière itération, ce qui peut ne pas toujours être idéal : cela signifie que de nombreux candidats fonctionneront avec toutes les ressources, réduisant ainsi essentiellement la procédure à une recherche standard.

Dans le cas de [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html), le nombre de candidats est défini par défaut de manière à ce que la dernière itération utilise autant de ressources disponibles que possible. Pour [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html), le nombre de candidats est déterminé par le paramètre `param_grid`. Le fait de modifier la valeur de `min_resources` aura un impact sur le nombre d'itérations possibles et, par conséquent, aura également un effet sur le nombre idéal de candidats.

Un autre facteur à prendre en compte lors du choix de `min_resources` est de savoir s'il est facile ou non de distinguer entre les bons et les mauvais candidats avec une petite quantité de ressources. Par exemple, si vous avez besoin de beaucoup d'échantillons pour distinguer entre de bons et de mauvais paramètres, il est recommandé d'utiliser une valeur élevée pour `min_resources`. En revanche, si la distinction est claire même avec une petite quantité d'échantillons, une petite valeur de `min_resources` peut être préférable car cela accélérerait le calcul.

Remarquez dans l'exemple ci-dessus que la dernière itération n'utilise pas la quantité maximale de ressources disponibles : 1000 échantillons sont disponibles, mais seuls 640 sont utilisés au maximum. Par défaut, à la fois [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) et [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) essaient d'utiliser autant de ressources que possible lors de la dernière itération, avec la contrainte que cette quantité de ressources doit être un multiple à la fois de `min_resources` et `factor` (cette contrainte sera expliquée dans la prochaine section). [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) atteint cet objectif en échantillonnant la bonne quantité de candidats, tandis que [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) y parvient en fixant correctement `min_resources`. Pour plus de détails, veuillez consulter [**Épuisement des ressources disponibles** (3.2.3.4)](https://scikit-learn.org/stable/modules/grid_search.html#exhausting-the-resources).

### <a id='amount-of-resource-and-number-of-candidates-at-each-iteration'></a> 3.2.3.2. Quantité de ressources et nombre de candidats à chaque itération

À chaque itération `i`, chaque candidat se voit allouer une quantité de ressources donnée que nous notons `n_resources_i`. Cette quantité est contrôlée par les paramètres `factor` et `min_resources` de la manière suivante (`factor` est strictement supérieur à 1) :

    n_resources_i = factor**i * min_resources

ou de manière équivalente :

    n_resources_{i+1} = n_resources_i * factor

où `min_resources == n_resources_0` représente la quantité de ressources utilisée lors de la première itération. `factor` définit également les proportions de candidats qui seront sélectionnés pour l'itération suivante :

    n_candidates_i = n_candidates // (factor ** i)

ou de manière équivalente :

    n_candidates_0 = n_candidates
    n_candidates_{i+1} = n_candidates_i // factor

Ainsi, lors de la première itération, nous utilisons `min_resources` ressources `n_candidates` fois. Lors de la deuxième itération, nous utilisons `min_resources * factor` ressources `n_candidates // factor` fois. Le processus se répète jusqu'à ce que la quantité maximale de ressources par candidat soit atteinte ou jusqu'à ce que nous ayons identifié le meilleur candidat. Le meilleur candidat est identifié lors de l'itération qui évalue `factor` candidats ou moins (voir ci-dessous pour une explication).

Voici un exemple avec `min_resources=3` et `factor=2`, en commençant avec 70 candidats :

| `n_resources_i` | `n_candidates_i` |
|-|-|
| 3 (=min_resources) | 70 (=n_candidates) |
| 3 * 2 = 6 | 70 // 2 = 35 |
| 6 * 2 = 12 | 35 // 2 = 17 |
| 12 * 2 = 24 | 17 // 2 = 8 |
| 24 * 2 = 48 | 8 // 2 = 4 |
| 48 * 2 = 96 | 4 // 2 = 2 |

Nous pouvons remarquer que :

- le processus s'arrête à la première itération qui évalue `factor=2` candidats : le meilleur candidat est le meilleur parmi ces 2 candidats. Il n'est pas nécessaire d'exécuter une itération supplémentaire, car elle n'évaluerait qu'un seul candidat (à savoir le meilleur, que nous avons déjà identifié). Pour cette raison, en général, nous voulons que la dernière itération évalue au plus `factor` candidats. Si la dernière itération évalue plus de `factor` candidats, alors cette dernière itération se réduit à une recherche classique (comme dans [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) ou [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).
- chaque `n_resources_i` est un multiple de `factor` et `min_resources` (ce qui est confirmé par sa définition ci-dessus).

La quantité de ressources utilisée à chaque itération peut être trouvée dans l'attribut `n_resources_`.

### <a id='choosing-a-resource'></a> 3.2.3.3. Choix d'une ressource

Par défaut, la ressource est définie en termes de nombre d'échantillons. C'est-à-dire que chaque itération utilisera un nombre croissant d'échantillons pour l'entraînement. Cependant, vous pouvez spécifier manuellement un paramètre à utiliser comme ressource avec le paramètre `resource`. Voici un exemple où la ressource est définie en termes du nombre d'estimateurs d'une forêt aléatoire :

In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
import pandas as pd

param_grid = {'max_depth': [3, 5, 10],
              'min_samples_split': [2, 5, 10]}
base_estimator = RandomForestClassifier(random_state=0)
X, y = make_classification(n_samples=1000, random_state=0)
sh = HalvingGridSearchCV(base_estimator, param_grid, cv=5,
                         factor=2, resource='n_estimators',
                         max_resources=30).fit(X, y)
sh.best_estimator_
# RandomForestClassifier(max_depth=5, n_estimators=24, random_state=0)

Veuillez noter qu'il n'est pas possible de fixer un budget sur un paramètre qui fait partie de la grille de paramètres.

### <a id='exhausting-the-available-resources'></a> 3.2.3.4. Épuisement des ressources disponibles

Comme mentionné ci-dessus, le nombre de ressources utilisées à chaque itération dépend du paramètre `min_resources`. Si vous disposez de nombreuses ressources disponibles mais commencez avec un faible nombre de ressources, certaines d'entre elles peuvent être gaspillées (c'est-à-dire non utilisées) :

In [ ]:
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
import pandas as pd
param_grid= {'kernel': ('linear', 'rbf'),
             'C': [1, 10, 100]}
base_estimator = SVC(gamma='scale')
X, y = make_classification(n_samples=1000)
sh = HalvingGridSearchCV(base_estimator, param_grid, cv=5,
                         factor=2, min_resources=20).fit(X, y)
sh.n_resources_
# [20, 40, 80]

Le processus de recherche n'utilisera au maximum que 80 ressources, alors que notre quantité maximale de ressources disponibles est `n_samples=1000`. Ici, nous avons `min_resources = r_0 = 20`.

Pour [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html), par défaut, le paramètre `min_resources` est défini sur ‘exhaust’. Cela signifie que `min_resources` est automatiquement défini de manière à ce que la dernière itération puisse utiliser autant de ressources que possible, dans la limite de `max_resources` :

In [ ]:
sh = HalvingGridSearchCV(base_estimator, param_grid, cv=5,
                         factor=2, min_resources='exhaust').fit(X, y)
sh.n_resources_
# [250, 500, 1000]

`min_resources` a été automatiquement défini ici sur 250, ce qui permet à la dernière itération d'utiliser toutes les ressources. La valeur exacte utilisée dépend du nombre de candidats paramètres, de `max_resources` et de `factor`.

Pour [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html), l'épuisement des ressources peut être réalisé de deux manières :
- en définissant `min_resources='exhaust'`, tout comme pour [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) ;
- en définissant `n_candidates='exhaust'`.

Ces deux options sont mutuellement exclusives : l'utilisation de `min_resources='exhaust'` nécessite de connaître le nombre de candidats, et symétriquement `n_candidates='exhaust'` nécessite de connaître `min_resources`.

En général, l'épuisement total des ressources conduit à un meilleur candidat final et prend un peu plus de temps.

### <a id='aggressive-elimination-of-candidates'></a> 3.2.3.5. Élimination agressive des candidats

Idéalement, nous souhaitons que la dernière itération évalue `factor` candidats (voir [**Quantité de ressources et nombre de candidats à chaque itération** (3.2.3.2)](https://scikit-learn.org/stable/modules/grid_search.html#amount-of-resource-and-number-of-candidates)). Il nous suffit ensuite de choisir le meilleur. Lorsque le nombre de ressources disponibles est faible par rapport au nombre de candidats, la dernière itération peut être amenée à évaluer plus de `factor` candidats :

In [ ]:
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
import pandas as pd
param_grid = {'kernel': ('linear', 'rbf'),
              'C': [1, 10, 100]}
base_estimator = SVC(gamma='scale')
X, y = make_classification(n_samples=1000)
sh = HalvingGridSearchCV(base_estimator, param_grid, cv=5,
                         factor=2, max_resources=40,
                         aggressive_elimination=False).fit(X, y)
sh.n_resources_
# [20, 40]
sh.n_candidates_
# [6, 3]

Comme nous ne pouvons pas utiliser plus de `max_resources=40` ressources, le processus doit s'arrêter à la deuxième itération, qui évalue plus de `factor=2` candidats.

En utilisant le paramètre `aggressive_elimination`, vous pouvez forcer le processus de recherche à se terminer avec moins de `factor` candidats lors de la dernière itération. Pour ce faire, le processus éliminera autant de candidats que nécessaire en utilisant `min_resources` ressources :

In [ ]:
sh = HalvingGridSearchCV(base_estimator, param_grid, cv=5,
                         factor=2,
                         max_resources=40,
                         aggressive_elimination=True,
                         ).fit(X, y)
sh.n_resources_
# [20, 20,  40]
sh.n_candidates_
# [6, 3, 2]

Remarquez que nous nous retrouvons avec 2 candidats lors de la dernière itération car nous avons éliminé suffisamment de candidats lors des premières itérations en utilisant `n_resources = min_resources = 20`.

### <a id='analyzing-results-with-the-cv-results-attribute'></a> 3.2.3.6. Analyse des résultats avec l'attribut `cv_results_`

L'attribut `cv_results_` contient des informations utiles pour analyser les résultats d'une recherche. Il peut être converti en un dataframe pandas avec `df = pd.DataFrame(est.cv_results_)`. L'attribut `cv_results_` de [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) et [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) est similaire à celui de [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) et [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html), avec des informations supplémentaires liées au processus de recherche successive halving.

Voici un exemple avec certaines des colonnes d'un dataframe (tronqué) :

|    | `iter` | `n_resources` | `mean_test_score` | `params`                                                                                   |
|----|------|-------------|-----------------|------------------------------------------------------------------------------------------|
| 0  | 0    | 125         | 0.983667        | {‘criterion’: ‘log_loss’, ‘max_depth’: None, ‘max_features’: 9, ‘min_samples_split’: 5}  |
| 1  | 0    | 125         | 0.983667        | {‘criterion’: ‘gini’, ‘max_depth’: None, ‘max_features’: 8, ‘min_samples_split’: 7}      |
| 2  | 0    | 125         | 0.983667        | {‘criterion’: ‘gini’, ‘max_depth’: None, ‘max_features’: 10, ‘min_samples_split’: 10}    |
| 3  | 0    | 125         | 0.983667        | {‘criterion’: ‘log_loss’, ‘max_depth’: None, ‘max_features’: 6, ‘min_samples_split’: 6}  |
| …  | …    | …           | …               | …                                                                                        |
| 15 | 2    | 500         | 0.951958        | {‘criterion’: ‘log_loss’, ‘max_depth’: None, ‘max_features’: 9, ‘min_samples_split’: 10} |
| 16 | 2    | 500         | 0.947958        | {‘criterion’: ‘gini’, ‘max_depth’: None, ‘max_features’: 10, ‘min_samples_split’: 10}    |
| 17 | 2    | 500         | 0.951958        | {‘criterion’: ‘gini’, ‘max_depth’: None, ‘max_features’: 10, ‘min_samples_split’: 4}     |
| 18 | 3    | 1000        | 0.961009        | {‘criterion’: ‘log_loss’, ‘max_depth’: None, ‘max_features’: 9, ‘min_samples_split’: 10} |
| 19 | 3    | 1000        | 0.955989        | {‘criterion’: ‘gini’, ‘max_depth’: None, ‘max_features’: 10, ‘min_samples_split’: 4}     |

Chaque ligne correspond à une combinaison de paramètres donnée (un candidat) et à une itération donnée. L'itération est donnée par la colonne `iter`. La colonne `n_resources` vous indique combien de ressources ont été utilisées.

Dans l'exemple ci-dessus, la meilleure combinaison de paramètres est `{'criterion': 'log_loss', 'max_depth': None, 'max_features': 9, 'min_samples_split': 10}` car elle a atteint la dernière itération (3) avec le score le plus élevé : 0.96.

### Références

🔬 [1] K. Jamieson, A. Talwalkar, [**“Non-stochastic Best Arm Identification and Hyperparameter Optimization”**](http://proceedings.mlr.press/v51/jamieson16.pdf), in proc. of Machine Learning Research, 2016.

🔬 [2] L. Li, K. Jamieson, G. DeSalvo, A. Rostamizadeh, A. Talwalkar, [**“Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization”**](https://arxiv.org/pdf/1603.06560.pdf), in Machine Learning Research 18, 2018.

## <a id='tips-for-parameter-search'></a> 3.2.4. Tips for parameter search

### <a id='specifying-an-objective-metric'></a> 3.2.4.1. Specifying an objective metric

By default, parameter search uses the `score` function of the estimator to evaluate a parameter setting. These are the [**`sklearn.metrics.accuracy_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score) for classification and [**`sklearn.metrics.r2_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) for regression. For some applications, other scoring functions are better suited (for example in unbalanced classification, the accuracy score is often uninformative). An alternative `scoring` function can be specified via the scoring parameter of most parameter search tools. See [**The scoring parameter: defining model evaluation rules** (3.3.1)](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) for more details.

### <a id='specifying-multiple-metrics-for-evaluation'></a> 3.2.4.2. Specifying multiple metrics for evaluation

[**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) allow specifying multiple metrics for the `scoring` parameter.

Multimetric scoring can either be specified as a list of strings of predefined scores names or a dict mapping the scorer name to the scorer function and/or the predefined scorer name(s). See [**Using multiple metric evaluation** (3.3.1.4)](https://scikit-learn.org/stable/modules/model_evaluation.html#multimetric-scoring) for more details.

When specifying multiple metrics, the `refit` parameter must be set to the metric (string) for which the `best_params_` will be found and used to build the `best_estimator_` on the whole dataset. If the search should not be refit, set `refit=False`. Leaving refit to the default value `None` will result in an error when using multiple metrics.

See [**Demonstration of multi-metric evaluation on `cross_val_score` and `GridSearchCV`**](https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py) for an example usage.

[**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) and [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) do not support multimetric scoring.

### <a id='composite-estimators-and-parameter-spaces'></a> 3.2.4.3. Composite estimators and parameter spaces

[**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) allow searching over parameters of composite or nested estimators such as [**`Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), [**`ColumnTransformer`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [**`VotingClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html) or [**`CalibratedClassifierCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html) using a dedicated `<estimator>__<parameter>` syntax:

---

Here, `<estimator>` is the parameter name of the nested `estimator`, in this case estimator. If the meta-estimator is constructed as a collection of estimators as in `pipeline.Pipeline`, then `<estimator>` refers to the name of the estimator, see [**Nested parameters** (6.1.1.1.3)](https://scikit-learn.org/stable/modules/compose.html#pipeline-nested-parameters). In practice, there can be several levels of nesting:

---

Please refer to [**Pipeline: chaining estimators** (6.1.1)](https://scikit-learn.org/stable/modules/compose.html#pipeline) for performing parameter searches over pipelines.

### <a id='model-selection-development-and-evaluation'></a> 3.2.4.4. Model selection: development and evaluation

Model selection by evaluating various parameter settings can be seen as a way to use the labeled data to “train” the parameters of the grid.

When evaluating the resulting model it is important to do it on held-out samples that were not seen during the grid search process: it is recommended to split the data into a **development set** (to be fed to the [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) instance) and an **evaluation set** to compute performance metrics.

This can be done by using the [**`train_test_split`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) utility function.

### <a id='parallelism'></a> 3.2.4.5. Parallelism

The parameter search tools evaluate each parameter combination on each data fold independently. Computations can be run in parallel by using the keyword `n_jobs=-1`. See function signature for more details, and also the Glossary entry for [`n_jobs`](https://scikit-learn.org/stable/glossary.html#term-n_jobs).

### <a id='robustness-to-failure'></a> 3.2.4.6. Robustness to failure

Some parameter settings may result in a failure to `fit` one or more folds of the data. By default, this will cause the entire search to fail, even if some parameter settings could be fully evaluated. Setting `error_score=0` (or `=np.nan`) will make the procedure robust to such failure, issuing a warning and setting the score for that fold to 0 (or `nan`), but completing the search

## <a id='tips-for-parameter-search'></a> 3.2.4. Conseils pour la recherche de paramètres

### <a id='specifying-an-objective-metric'></a> 3.2.4.1. Spécification d'une métrique objective

Par défaut, la recherche de paramètres utilise la fonction `score` de l'estimateur pour évaluer un paramètre donné. Il s'agit des fonctions [**`sklearn.metrics.accuracy_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score) pour la classification et [**`sklearn.metrics.r2_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) pour la régression. Pour certaines applications, d'autres fonctions de score sont mieux adaptées (par exemple, dans la classification déséquilibrée, le score d'exactitude n'est souvent pas informatif). Une fonction `scoring` alternative peut être spécifiée via le paramètre `scoring` de la plupart des outils de recherche de paramètres. Consultez [**Le paramètre de score : définition des règles d'évaluation du modèle** (3.3.1)](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) pour plus de détails.

### <a id='specifying-multiple-metrics-for-evaluation'></a> 3.2.4.2. Spécification de plusieurs métriques pour l'évaluation

[**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) et [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) permettent de spécifier plusieurs métriques pour le paramètre `scoring`.

L'évaluation multi-métrique peut être spécifiée soit sous forme d'une liste de chaînes de noms de scores prédéfinis, soit sous forme d'un dictionnaire associant le nom du score à la fonction de score et/ou aux noms de scores prédéfinis. Consultez [**Utilisation de l'évaluation multi-métrique** (3.3.1.4)](https://scikit-learn.org/stable/modules/model_evaluation.html#multimetric-scoring) pour plus de détails.

Lors de la spécification de plusieurs métriques, le paramètre `refit` doit être défini sur la métrique (chaîne de caractères) pour laquelle les `best_params_` seront trouvés et utilisés pour construire le `best_estimator_` sur l'ensemble du jeu de données. Si la recherche ne doit pas être refaite, définissez `refit=False`. Laisser la valeur par défaut `None` pour le paramètre `refit` entraînera une erreur lors de l'utilisation de plusieurs métriques.

Veuillez consulter [**Démonstration de l'évaluation multi-métrique sur `cross_val_score` et `GridSearchCV`**](https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py) pour un exemple d'utilisation.

[**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) et [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) ne prennent pas en charge l'évaluation multi-métrique.

### <a id='composite-estimators-and-parameter-spaces'></a> 3.2.4.3. Estimateurs composites et espaces de paramètres

[**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) et [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) permettent la recherche de paramètres d'estimateurs composites ou imbriqués tels que [**`Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), [**`ColumnTransformer`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [**`VotingClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html) ou [**`CalibratedClassifierCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html) en utilisant une syntaxe dédiée `<estimateur>__<paramètre>` :

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_moons
X, y = make_moons()
calibrated_forest = CalibratedClassifierCV(
    estimator=RandomForestClassifier(n_estimators=10))
param_grid = {'estimator__max_depth': [2, 4, 6, 8]}
search = GridSearchCV(calibrated_forest, param_grid, cv=5)
search.fit(X, y)
# GridSearchCV(cv=5,
#              estimator=CalibratedClassifierCV(...),
#              param_grid={'estimator__max_depth': [2, 4, 6, 8]})

GridSearchCV(cv=5,
             estimator=CalibratedClassifierCV(estimator=RandomForestClassifier(n_estimators=10)),
             param_grid={'estimator__max_depth': [2, 4, 6, 8]})

Ici, `<estimateur>` est le nom du paramètre de l'estimateur imbriqué, dans ce cas, l'estimateur. Si le méta-estimateur est construit comme une collection d'estimateurs comme dans `pipeline.Pipeline`, alors `<estimateur>` se réfère au nom de l'estimateur, voir [**Paramètres imbriqués** (6.1.1.1.3)](https://scikit-learn.org/stable/modules/compose.html#pipeline-nested-parameters). En pratique, il peut y avoir plusieurs niveaux d'imbrication :

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
pipe = Pipeline([
    ('select', SelectKBest()),
    ('model', calibrated_forest)])
param_grid = {
    'select__k': [1, 2],
    'model__estimator__max_depth': [2, 4, 6, 8]}
search = GridSearchCV(pipe, param_grid, cv=5).fit(X, y)

Veuillez vous référer à [**Pipeline : enchaînement d'estimateurs** (6.1.1)](https://scikit-learn.org/stable/modules/compose.html#pipeline) pour effectuer des recherches de paramètres sur les pipelines.

### <a id='model-selection-development-and-evaluation'></a> 3.2.4.4. Sélection de modèle : développement et évaluation

La sélection de modèle en évaluant diverses valeurs de paramètres peut être considérée comme une façon d'utiliser les données étiquetées pour "entraîner" les paramètres de la grille.

Lors de l'évaluation du modèle résultant, il est important de le faire sur des échantillons de données non vus lors du processus de recherche de la grille : il est recommandé de diviser les données en un **ensemble de développement** (à alimenter à l'instance [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)) et un **ensemble d'évaluation** pour calculer les métriques de performance.

Cela peut être fait en utilisant la fonction utilitaire [**`train_test_split`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

### <a id='parallelism'></a> 3.2.4.5. Parallélisme

Les outils de recherche de paramètres évaluent chaque combinaison de paramètres sur chaque pli de données indépendamment. Les calculs peuvent être exécutés en parallèle en utilisant le mot-clé `n_jobs=-1`. Consultez la signature de la fonction pour plus de détails, ainsi que l'entrée du glossaire pour [`n_jobs`](https://scikit-learn.org/stable/glossary.html#term-n_jobs).

### <a id='robustness-to-failure'></a> 3.2.4.6. Robustesse face aux échecs

Certaines valeurs de paramètres peuvent entraîner une défaillance de `fit` pour un ou plusieurs plis des données. Par défaut, cela entraînera l'échec de toute la recherche, même si certaines valeurs de paramètres pourraient être entièrement évaluées. En définissant `error_score=0` (ou `=np.nan`), la procédure deviendra robuste à de tels échecs, émettra un avertissement et définira le score pour ce pli à 0 (ou `nan`), mais terminera la recherche sur les autres plis avec succès.

## <a id='alternatives-to-brute-force-parameter-search'></a> 3.2.5. Alternatives à la recherche de paramètres par force brute

### <a id='model-specific-cross-validation'></a> 3.2.5.1. Validation croisée spécifique au modèle

Certains modèles peuvent ajuster les données pour une gamme de valeurs d'un paramètre presque aussi efficacement que pour une seule valeur de ce paramètre. Cette caractéristique peut être exploitée pour effectuer une validation croisée plus efficace utilisée pour la sélection du modèle de ce paramètre.

Le paramètre le plus courant pouvant bénéficier de cette stratégie est le paramètre codant la force de la régularisation. Dans ce cas, on dit que nous calculons le **chemin de régularisation** de l'estimateur.

Voici la liste de ces modèles :

| Modèle                                             | Description                                                             |
|----------------------------------------------------|-------------------------------------------------------------------------|
| [**`linear_model.ElasticNetCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html)(*[, l1_ratio, ...])     | Modèle Elastic Net avec ajustement itératif le long d'un chemin de régularisation.  |
| [**`linear_model.LarsCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LarsCV.html)(*[, fit_intercept, ...])      | Modèle de régression du meilleur angle de croisement validée.            |
| [**`linear_model.LassoCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html)(*[, eps, n_alphas, ...])     | Modèle linéaire Lasso avec ajustement itératif le long d'un chemin de régularisation. |
| [**`linear_model.LassoLarsCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsCV.html)(*[, fit_intercept, ...]) | Lasso validé par croisement, en utilisant l'algorithme LARS.                       |
| [**`linear_model.LogisticRegressionCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html)(*[, Cs, ...])   | Classifieur de régression logistique validé par croisement (également appelé logit, MaxEnt).                 |
| [**`linear_model.MultiTaskElasticNetCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.MultiTaskElasticNetCV.html)(*[, ...])      | Multi-tâches L1/L2 ElasticNet avec validation croisée intégrée.            |
| [**`linear_model.MultiTaskLassoCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.MultiTaskLassoCV.html)(*[, eps, ...])      | Modèle Lasso multi-tâches entraîné avec la norme mixte L1/L2 comme régularisateur, et validé par croisement.   |
| [**`linear_model.OrthogonalMatchingPursuitCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.OrthogonalMatchingPursuitCV.html)(*)       | Modèle de poursuite orthogonale validée par croisement (OMP).               |
| [**`linear_model.RidgeCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)([alphas, ...])               | Régression Ridge avec validation croisée intégrée.                       |
| [**`linear_model.RidgeClassifierCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifierCV.html)([alphas, ...])     | Classifieur Ridge avec validation croisée intégrée.                        |


### <a id='information-criterion'></a> 3.2.5.2. Critère d'information

Certains modèles peuvent offrir une formule fermée d'estimation optimale du paramètre de régularisation en utilisant un seul chemin de régularisation (au lieu de plusieurs avec la validation croisée).

Voici la liste des modèles bénéficiant du critère d'information d'Akaike (AIC) ou du critère d'information bayésien (BIC) pour la sélection automatisée du modèle :

| Modèle                                             | Description                                                            |
|----------------------------------------------------|------------------------------------------------------------------------|
| [**`linear_model.LassoLarsIC`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsIC.html)([*criterion*, ...])       | Modèle Lasso ajusté avec Lars en utilisant BIC ou AIC pour la sélection du modèle.        |

### <a id='out-of-bag-estimates'></a> 3.2.5.3. Estimations out-of-bag

Lors de l'utilisation de méthodes d'ensemble basées sur le bagging, c'est-à-dire générant de nouveaux ensembles d'entraînement en utilisant l'échantillonnage avec remplacement, une partie de l'ensemble d'entraînement reste inutilisée. Pour chaque classifieur de l'ensemble, une partie différente de l'ensemble d'entraînement est laissée de côté.

Cette partie laissée de côté peut être utilisée pour estimer l'erreur de généralisation sans avoir à utiliser un ensemble de validation distinct. Cette estimation est obtenue "gratuitement" car aucune donnée supplémentaire n'est nécessaire et peut être utilisée pour la sélection du modèle.

Cette fonctionnalité est actuellement implémentée dans les classes suivantes :

| Modèle                                             | Description                                                            |
|----------------------------------------------------|------------------------------------------------------------------------|
| [**`ensemble.RandomForestClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)([...]) | Un classifieur random forest.                                           |
| [**`ensemble.RandomForestRegressor`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)([...]) | Un régresseur random forest.                                           |
| [**`ensemble.ExtraTreesClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html)([...]) | Un classifieur extra-trees.                                            |
| [**`ensemble.ExtraTreesRegressor`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html)([n_estimators, ...]) | Un régresseur extra-trees.                                           |
| [**`ensemble.GradientBoostingClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)(*[, ...]) | Gradient Boosting pour la classification.                                 |
| [**`ensemble.GradientBoostingRegressor`**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)(*[, ...]) | Gradient Boosting pour la régression.                                     |